# 🌾 CSIRO Image2Biomass: Data Augmentation and Training Notebook

**Competition:** [CSIRO Image2Biomass Prediction](https://www.kaggle.com/competitions/csiro-biomass)

**Thank you CigarCat for the useful EDA** [NB link](https://www.kaggle.com/code/takahitomizunobyts/csiro-customizable-eda)

**Thank you Zhuang Jia for some of the albumentations** [NB link](https://www.kaggle.com/code/jiazhuang/csiro-simple)

**Thank you Sagar Nagpure for the DINO baseline model**[NB link](https://www.kaggle.com/code/sagarnagpure1310/csiro-dinov2-tiled-lb-0-65)

---

## 📋 **Notebook Overview**

This notebook provides **model and data preprocessing** for the CSIRO Image2Biomass competition.

### **Competition Context:**

**Goal:** Predict 5 biomass components from pasture images:
- `Dry_Green_g` (10% weight)
- `Dry_Dead_g` (10% weight)
- `Dry_Clover_g` (10% weight)
- `GDM_g` (20% weight)
- `Dry_Total_g` (50% weight) ⭐

**Evaluation:** Weighted R² score


From the EDA in the other notebook, we found out that:
- The data is right-skewed and there are several outliers (high kurtosis)
- Vegetation is dense, but some samples are sparser due to the data distribution
- NDVI and height are positively correlated with biomass

We need to perform data augmentation and handle outliers

The primary goal is to predict five biomass targets. Based on exploratory data analysis (EDA), we identified linear dependencies:

Dry_Total_g 
≈
 Dry_Green_g + Dry_Dead_g + Dry_Clover_g
GDM_g 
≈
 Dry_Green_g + Dry_Clover_g

To avoid redundancy, the model is trained to predict only the three most visually distinct and/or highest-weighted targets:

- Dry_Total_g (50% of the score)
- GDM_g (20% of the score)
- Dry_Green_g (10% of the score)

The remaining two targets (Dry_Dead_g and Dry_Clover_g) are then calculated during validation and inference using subtraction (e.g., pred_Clover = max(0, pred_GDM - pred_Green)).

## 1. Setup, setting globals

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
from scipy import stats
import warnings

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from torch.utils.data import Dataset
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Libraries imported successfully")

In [ ]:
# Configuration
CSV_PATH_TRAIN = "./csiro-biomass/train.csv"
CSV_PATH_TEST = "./csiro-biomass/test.csv"
IMAGE_DIR = "./csiro-biomass"

# Target columns
TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']

# Competition weights
WEIGHTS = {
    'Dry_Green_g': 0.1,
    'Dry_Dead_g': 0.1,
    'Dry_Clover_g': 0.1,
    'GDM_g': 0.2,
    'Dry_Total_g': 0.5
}

# Random seed for reproducibility
RANDOM_SEED = 42

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Configuration set")

## 2. Data loading and preprocessing

In [ ]:
def load_and_preprocess_data(csv_path: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Load and preprocess biomass data
    
    Returns:
        Tuple of (long_format_df, wide_format_df)
    """
    # Load data
    print("Loading data...")
    df_long = pd.read_csv(csv_path)
    print(f"Loaded {len(df_long):,} rows × {len(df_long.columns)} columns")
    
    # Extract image ID
    df_long['image_id'] = df_long['sample_id'].str.split('__').str[0]
    
    # Convert to wide format
    print("\n🔄 Converting to wide format...")
    meta_cols = ['image_path', 'Sampling_Date', 'State', 'Species', 
                 'Pre_GSHH_NDVI', 'Height_Ave_cm']
    
    df_wide = df_long.pivot_table(
        index=['image_id'] + meta_cols,
        columns='target_name',
        values='target',
        aggfunc='first'
    ).reset_index()
    
    n_images = len(df_wide)
    print(f"Wide format: {n_images:,} images")
    
    # Add temporal features
    print("\nAdding temporal features...")
    df_wide['date'] = pd.to_datetime(df_wide['Sampling_Date'], format='%Y/%m/%d')
    df_wide['year'] = df_wide['date'].dt.year
    df_wide['month'] = df_wide['date'].dt.month
    df_wide['day'] = df_wide['date'].dt.day
    df_wide['dayofweek'] = df_wide['date'].dt.dayofweek
    df_wide['dayofyear'] = df_wide['date'].dt.dayofyear
    
    # Add season (Southern Hemisphere)
    def get_season(month: int) -> str:
        if month in [9, 10, 11]:
            return "Spring"
        elif month in [12, 1, 2]:
            return "Summer"
        elif month in [3, 4, 5]:
            return "Autumn"
        else:
            return "Winter"
    
    df_wide['season'] = df_wide['month'].apply(get_season)
    
    # Add derived features
    print("Adding derived features...")
    
    # Component ratios
    total = df_wide['Dry_Total_g'] + 1e-8 # Avoid division by 0
    df_wide['green_ratio'] = df_wide['Dry_Green_g'] / total
    df_wide['dead_ratio'] = df_wide['Dry_Dead_g'] / total
    df_wide['clover_ratio'] = df_wide['Dry_Clover_g'] / total
    
    # Physical constraint check
    df_wide['sum_components'] = (
        df_wide['Dry_Green_g'] + 
        df_wide['Dry_Dead_g'] + 
        df_wide['Dry_Clover_g']
    )
    df_wide['constraint_diff'] = df_wide['Dry_Total_g'] - df_wide['sum_components']
    df_wide['constraint_error'] = np.abs(df_wide['constraint_diff'])
    
    print("Preprocessing complete!\n")
    
    return df_long, df_wide

# Execute
df_long, df = load_and_preprocess_data(CSV_PATH_TRAIN)

### Transforming data to reduce skewness and outliers

We apply the Yeo-Johnson transformation to make data more normally distributed and stabilize variance 

In [ ]:
from sklearn.preprocessing import PowerTransformer 

numeric_features = [
    "Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g",
    "Pre_GSHH_NDVI", "Height_Ave_cm", "green_ratio", "dead_ratio", "clover_ratio",
    "constraint_error"
]
categorical_features = ["State", "Species", "season"]


In [ ]:
import albumentations as A # This works better for cpu-bound processing
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np

# Training augmentations
train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0), p=1.0),
    A.GaussianBlur(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), 
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Validation/test transforms (no augmentation, just resize + normalize)
# TODO investigate test time augmentations!!
val_transforms = A.Compose([
    A.Resize(height=256, width=256),
    A.Normalize(mean=(0.485, 0.456, 0.406), 
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

class BiomassDataset(Dataset):
    def __init__(self, df, numeric_features, categorical_features,
                 target_cols=TARGET_COLS, train=True, scaler=None, encoder=None,
                 transform=None):
        self.df = df.copy()
        self.train = train
        self.transform = transform

        # Drop train-only features if not training
        if not train:
            numeric_features = [f for f in numeric_features if f not in ["Pre_GSHH_NDVI", "Height_Ave_cm"]]
            categorical_features = [f for f in categorical_features if f not in ["State", "Species"]]

        # Scale numeric features
        if scaler is None:
            self.pt = PowerTransformer(method="yeo-johnson") # Data is not strictly positive :c, yeo-johnson generalizes box-cox
            self.df[numeric_features] = self.pt.fit_transform(self.df[numeric_features])

            self.scaler = RobustScaler()
            self.df[numeric_features] = self.scaler.fit_transform(self.df[numeric_features])
        else:
            self.pt = scaler[0]
            self.scaler = scaler[1]
            self.df[numeric_features] = self.pt.transform(self.df[numeric_features])
            self.df[numeric_features] = self.scaler.transform(self.df[numeric_features])

        # Encode categorical features
        if categorical_features:
            if encoder is None:
                self.encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
                encoded = self.encoder.fit_transform(self.df[categorical_features])
            else:
                self.encoder = encoder
                encoded = self.encoder.transform(self.df[categorical_features])
            encoded_df = pd.DataFrame(encoded, index=self.df.index)
            self.features = np.hstack([self.df[numeric_features].values, encoded_df.values])
        else:
            self.encoder = None
            self.features = self.df[numeric_features].values

        self.targets = self.df[target_cols].values.astype(np.float32)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Tabular features
        x_tab = torch.tensor(self.features[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.float32)

        # Image features
        img_path = self.df.iloc[idx]["image_path"]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return (x_tab, image), y

In [ ]:
#print tabular features dimensions
bm = BiomassDataset(df, numeric_features, categorical_features, train=True)
print(f"Number of tabular features: {bm.features.shape[1]}")

### Using PyTorch Lightning

In [ ]:
from sklearn.model_selection import train_test_split

class BiomassDataModule(pl.LightningDataModule):
    def __init__(self, df, numeric_features, categorical_features, target_cols=TARGET_COLS, batch_size=32):
        super().__init__()
        self.df = df
        self.numeric_features = numeric_features
        self.categorical_features = categorical_features
        self.target_cols = target_cols
        self.batch_size = batch_size

    def setup(self, stage=None):
        # TODO see if this method of splitting is okay or if stratification is needed
        train_df, val_df = train_test_split(self.df, test_size=0.2, random_state=RANDOM_SEED)

        self.train_dataset = BiomassDataset(train_df, self.numeric_features,
                                            self.categorical_features, self.target_cols,
                                            train=True, transform=train_transforms)

        self.val_dataset = BiomassDataset(val_df, self.numeric_features,
                                          self.categorical_features, self.target_cols,
                                          train=False, transform=val_transforms)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False)

In [ ]:
dm = BiomassDataModule(df, numeric_features, categorical_features)
dm.setup()

train_loader = dm.train_dataloader()
print(f"Train DataModule setup complete!")
val_loader = dm.val_dataloader()
print(f"Validation DataModule setup complete!")

In [ ]:
print(f"Number of training samples: {len(dm.train_dataset)}, percentage: {len(dm.train_dataset)/len(df)*100:.2f}%")
print(f"Number of validation samples: {len(dm.val_dataset)}, percentage: {len(dm.val_dataset)/len(df)*100:.2f}%")

#print tabular features dimensions
print(f"Number of tabular features: {dm.train_dataset.features.shape[1]}") 

## Baseline model definition

Fetching a pretrained version from timm and wrapping it in Pytorch Lightning

Trying to use DINO

In [ ]:
import timm

DINO_CANDIDATES = [
    "vit_base_patch14_dinov3",
    "vit_base_patch14_reg4_dinov3",
    "vit_small_patch14_dinov3",
    "vit_base_patch14_reg4_dinov2",
    "vit_base_patch14_dinov2",
    "vit_small_patch14_dinov2",
]

class BiomassModel(pl.LightningModule):
    def __init__(self, tabular_input_dim, target_dim=len(TARGET_COLS), dino_model_name="vit_base_patch14_dinov3", lr=1e-4):
        super().__init__()
        self.save_hyperparameters()

        # Load pre-trained DINO model
        self.dino = timm.create_model(dino_model_name, pretrained=True)
        self.dino.head = torch.nn.Identity()  # Remove classification head

        dino_output_dim = self.dino.num_features

        # Tabular data branch
        self.tabular_branch = torch.nn.Sequential(
            torch.nn.Linear(tabular_input_dim, 128),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(128),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(64),
        )

        # Combined regression head
        self.regression_head = torch.nn.Sequential(
            torch.nn.Linear(dino_output_dim + 64, 256),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(256),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(256, target_dim)
        )

        self.criterion = torch.nn.MSELoss()
        self.lr = lr

    def forward(self, x_tab, x_img):
        # Image branch
        img_features = self.dino(x_img)

        # Tabular branch
        tab_features = self.tabular_branch(x_tab)

        # Concatenate features
        combined_features = torch.cat((img_features, tab_features), dim=1)

        # Regression head
        outputs = self.regression_head(combined_features)
        return outputs
    
    def training_step(self, batch, batch_idx):
        (x_tab, x_img), y = batch
        y_pred = self(x_tab, x_img)
        loss = self.criterion(y_pred, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        (x_tab, x_img), y = batch
        y_pred = self(x_tab, x_img)
        loss = self.criterion(y_pred, y)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss'
            }
        }

In [ ]:
# Trainer instance with best checkpoint saving and early stopping
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./checkpoints',
    filename="best-model-{epoch:02d}-{val_loss:.2f}",  # custom filename
    save_top_k=1,
    mode='min'
)

earlystopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',
    verbose=True
)

trainer = pl.Trainer(
    max_epochs=50,
    accelerator='auto',
    callbacks=[checkpoint_callback, 
               earlystopping_callback,
               ],
    log_every_n_steps=10
)

In [ ]:
# Instantiate model
tabular_input_dim = dm.train_dataset.features.shape[1]
model = BiomassModel(tabular_input_dim=tabular_input_dim, dino_model_name="vit_base_patch14_dinov3", lr=1e-4)
print("Model instantiated successfully")

# Train the model
# TODO add cross-validation
trainer.fit(model, dm)
print("Training complete!")